## Installing SCAPE

``` bash
## This code is run in command line

## Create virtual environment and install scape
conda create --name myscape
conda activate myscape

## Install taichi before installing SCAPE-APA
pip install taichi
pip install scape-apa

## Test if the latest SCAPE-APA is installed successfully
scape --help
scape gen_utr_annotation --help
```

__Example of outputs from SCAPE__

```python
output_dir:
    - utr.csv
    - pkl_input/ 
        + bam_stem.100.14.1.input.pkl
    - pkl_output/
        + bam_stem.100.14.1.res.pkl
    - res.gene.pkl or res.utr.pkl (after merge_pa)
    - user_stem.gene.pa.len.csv or user_stem.utr.pa.len.csv
    - res.utr.cnt.tsv.gz or res.gene.cnt.tsv.gz
```

## Prepare UTR

This step will derive potential UTRs from provided annotation file. This annotation file should be identical or close to the annotation file used in alignment step.

```bash
## This code is run in bash
scape gen_utr_annotation \
--gff_file Homo_sapiens.GRCh38.98.chr.gff3.gz \
--output_dir /your-path/scape/examples/toy-example \
--res_file_name GRCh38_98 \
--gff_merge_strategy merge
```



__INPUT__
- `gff_file`: The gff3 or gff3.gz file including annotation of gene body.
- `output_dir`: Directory to save dataframe of selected UTR.
- `res_file_name`: File Name of dataframe of the UTR annotation. The suffix ".csv" is automatically generated.
- `gff_merge_strategy`: Method for processing overlapping regions. It follows `merge_strategy` in package _gffutils_. The default value is _merge_.

__OUTPUT__
An csv file including information of annotated 3UTR which is stored at {output_dir}/{res_file_name}.csv:
- `chrom`: chromosome name
- `start`: position of the leftmost of 3UTR
- `end`: position of the rightmost of 3UTR
- `strand`: strand
- `gene_id`: ensembl gene ID
- `gene_name`: name of gene
- `utr_id`: ID of UTR per gene




A warning as following can be ignored.

```bash
You have installed a restricted version of taichi, certain features (e.g. Vulkan & GGUI) will not work.
!! Taichi requires glibc >= 2.27 to run, please try upgrading your OS to a recent one (e.g. Ubuntu 18.04 or later) if possible.
You can suppress this warning by setting the environment variable TI_MANYLINUX2014_OK=1.
[W 07/19/23 20:36:39.627 8936] [cuda_driver.cpp:load_lib@36] libcuda.so lib not found.
[W 07/19/23 20:36:39.641 8936] [misc.py:adaptive_arch_select@753] Arch=[<Arch.cuda: 4>] is not supported, falling back to CPU
```

In this example, we get the following output file `GRCh38_98.csv` in the output directory `/your-path/scape/examples/toy-example`. The file looks like the following:

```python
import pandas as pd

df=pd.read_csv("/toy-example/example-result/GRCh38_98.csv")
df.head()

>>>
  chrom   start     end strand          gene_id gene_name  utr_id
0     1   69709   71885      +  ENSG00000186092     OR4F5       1
1     1  450403  451039      -  ENSG00000284733    OR4F29       1
2     1  685379  686015      -  ENSG00000284662    OR4F16       1
3     1  935472  936093      +  ENSG00000187634    SAMD11       1
4     1  938975  939591      +  ENSG00000187634    SAMD11       2
```

## Prepare data per UTR

This step will process BAM files by filtering in high-quality reads that are mapped to UTRs defined in the above step. It also only considers reads that are corresponding to valid barcodes defined in alignment step.

```bash
## this code is run in bash
scape prepare_input \
--utr_file /your-scape/examples/toy-example/GRCh38_98.csv \
--cb_file /your-scape/examples/toy-example/barcodes.tsv.gz \
--bam_file /your-scape/examples/toy-example/example.bam \
--output_dir /your-scape/examples/toy-example \
--chunksize 100
```



__INPUT__

- `utr_file`: UTR annotation file (dataframe, resulted from gen_utr_annotation).
- `cb_file`: File of tsv.gz including all validated barcodes (by CellRanger). This file has one column of cell barcode which must be consistent with value of CB tag in bam_file file.
- `bam_file`: Bam file that is used for searching reads over annotated UTR.
- `output_dir`: Directory to save pickle files of selected reads over annotated UTR.
- `chunksize`: Number of UTR regions included in each small pickle file, which contains preprocessed input file for APA analysis. Default value is 1000.

__OUTPUT__
- A csv file includes 2 columns index and CB (barcode). It is stored in `output_dir/barcode_index.csv`. Index is generated by `prepare_input` and needed when converting SCAPE result to read count matrix.


- A folder `output_dir/pkl_input/` is generated to store pickle files which are containing each element (utr_id, dataframe df(x,l,r,pa, cb_id, read_id, junction, seg1_en, seg2_en)). By default, each pickle file includes 1000 utr regions. The output files are storded in `{output_dir}/pkl_input/{prefix}.input.pkl`
    - `x`: position of the 5'-most of aligned read
    - `l`: relative position of the leftmost of aligned read
    - `r`: relative position of the rightmost of aligned read
    - `pa`: length of consecutive sequence of A at the 3'-most
    - `cb_id`: index of cell barcode generated by prepare_input()
    - `read_id`: index of read generated by prepare_input()
    - `junction`: Flag equal to 1 means thats there exists junction site in this read
    - `seg1_en`: position of the 3'-most of sequence on the left of junction site
    - `seg2_en`: position of the 3'-most of sequence on the right of junction site

- A log file to check if given BAM file was successfully processed in `prepare_input()`. 



__In this example__, an csv file `barcode_index.csv` is generated in the output directory `/your-path/examples/toy-example`. And pickle file (`example.100.1.1.input.pkl`) is generated in the folder `pkl_input` in the output directory `/your-path/examples/toy-example`

In `example.100.1.1.input.pkl`, 

- `example` is the BAM file name
- `100.1.1` = `chunksize` __.__ `total number of pickle files for example.bam` __.__ `order of the pickle file`

The file looks like following:

```python
## This code is run in Python
import pickle
with open("/your-scape/examples/toy-example/pkl_input/example.100.1.1.input.pkl", "rb") as f_h:
    a=pickle.load(f_h)

a
>>>
('10:ENSG00000099194:1:100360634-100365126:+',           x    l   r      pa  cb_id  read_id  junction  seg1_en  seg2_en
0        50   98 NaN     NaN  23662        0         0      NaN      NaN
1       137   93 NaN     NaN   2134        1         0      NaN      NaN
2        99   98 NaN     NaN  20178        2         0      NaN      NaN
3        99   98 NaN     NaN  24870        3         0      NaN      NaN
4        99  132 NaN     NaN  24315        4         0      NaN      NaN
...     ...  ...  ..     ...    ...      ...       ...      ...      ...
27824  4184   37 NaN  4220.0   9300    27824         0      NaN      NaN
27825  4184   37 NaN  4220.0   8471    27825         0      NaN      NaN
27826  4184   36 NaN  4219.0  31564    27826         0      NaN      NaN
27827  4185   36 NaN  4220.0   8562    27827         0      NaN      NaN
27828  4188   33 NaN  4220.0  20698    27828         0      NaN      NaN

[27829 rows x 9 columns])
```

## SCAPE inference

This step will derive position of PA sites from given inputs generated in the above steps.

Function `infer_pa` need to be applied to each input pickle file separately.

```bash
## This code is run in bash
scape infer_pa \
--pkl_input_file /your-scape/examples/toy-example/pkl_input/example.100.1.1.input.pkl \
--output_dir /your-scape/examples/toy-example
```




__INPUT__

- `input_pickle_file`: full path of one `prefix.input.pkl` 
- `output_dir`: Directory to save output pickle files including PAS information over annotated UTR.

__OUTPUT__

A folder `output_dir/pkl_output/` is generated to store res pickle files including Parameters objects. 

__In this example__, many pickles files (ex: GSE1848781_chr17.100.1.1.res.pkl) is generated in the folder `pkl_output` in the output directory `/your-path/examples/toy-example`.

A object includes information:

```python
## This code is run in python
## How to display Parameters object
import pickle

## Load result object
with open("/your-scape/examples/toy-example/pkl_output/example.100.1.1.res.pkl", "rb") as f_h:
    a=pickle.load(f_h)
    
## String of result object
>>> print(a)
----------Final Result K=2----------
gene info: 10:ENSG00000099194:1:100360634-100365126:+
K=2 L=4370 Last component is uniform component.
alpha_arr=[2965 4171]
beta_arr=[45. 50.]
ws=[0.15 0.8  0.05]
bic=894989.65
------------------------------

## Attributes of result object
>>> print(dir(a))
['K', 'L', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'alpha_arr', 'beta_arr', 'bic', 'cb_id_arr', 'gene_info_str', 'label_arr', 'lb_arr', 'readID_arr', 'title', 'ws']
```

- `label_arr` : Array of pa id respective to each read. Its value starts from 0. Value K is for uniform component.
- `K`: The total number of detected PASs by SCAPE.
- `ws`: Proportion of reads per PAS in this UTR region.

The absolute position of PAS is calculated as following:
 - ```"-" strand -> pa position = right of utr - alpha + 1```
 - ```"+" strand -> pa position = left of utr + alpha```
 

## Merge PA within junction per gene or utr

Since there might be spurious PA sites during inference step (for example, due to splicing sites), this step will exclude those with significant evidence of present of splicing sites.

```bash
## This code is run in bash

scape merge_pa \
--output_dir /your-scape/examples/toy-example \
--utr_merge True

scape merge_pa \
--output_dir /your-scape/examples/toy-example \
--utr_merge False
```

__INPUT__
- `output_dir`: Directory which was used in previous steps to save output by `prepare_input` and `infer_pa`
- `utr_merge`: If `True`, PA sites from the same gene are merge. Otherwise, if `False`, PA sites from the same UTR are merged. Default value is `True`

__OUTPUT__

A single pickle file containing all UTRs of all genes is stored in `output_dir/`. Its name is `res.gene.pkl` if `utr_merge=True`, otherwise, its name is `res.utr.pkl`.

__In this example__, the first command generated `res.gene.pkl`, while the seconde command generated `res.utr.pkl`. Both are in the output directory `/your-scape/examples/toy-example`.


## Calculate expected length of PA

This step will calculate the expected length of PA in specific group of cells. Group of cells will be defined in a csv file including two information: index of cell barcode, and corresponding label of cell. The first information is generated during the second step (`scape prepare_input`). While the second information is provided by user (for example: cluster labels, cell types). 

The expected length of PA can be used to identify abnormal behavior in expression of different PA sites of the same gene (or UTR) in different conditions.


```bash
## This code is run in bash
scape cal_exp_pa_len \
--output_dir /your-scape/examples/toy-example \
--cell_cluster_file /your-scape/examples/toy-example/cluster_wrt_CB.csv \
--res_pkl_file res.gene.pkl


scape cal_exp_pa_len \
--output_dir /your-scape/examples/toy-example \
--cell_cluster_file /your-scape/examples/toy-example/cluster_wrt_CB.csv \
--res_pkl_file res.utr.pkl

```

__INPUT__
- `output_dir`: Directory which was used in previous steps to save output by `prepare_input` and `infer_pa`
- `cell_cluster_file`: An csv file containing two columns in order: cell barcode (CB) and respective group (cell_cluster_file). Its name will be included in the file name of final result.
- `res_pkl_file`: Name of res pickle file that contains PASs for calculating expected PA length. Its name will be included in the file name of final result.

__OUTPUT__
An csv file named `{cell_cluster_file}.{gene or utr}.pa.len.csv` is stored in `output_dir/`. It contains four columns:
- `gene_id`: Value is in format `GeneID:utrID`. `utrID` is always equal to 1 when `res.gene.pkl` is used as input. It means that expected PA length is calculated for each gene. Otherwise, `res.utr.pkl`, it means that expected PA length is calculated separately for each UTR of each gene and `utrID` can be different from 1.
- `cell_cluster`: Pre-defined label of each cell in `{cell_cluster_file}`. It means that expected PA length is calculated using cells from each cell label.  
- `expected_length`: Calculated expected PA length. 
- `num_pa`: number of PAS in each gene (if `res.gene.pkl` as input) or each UTR of each gene ( if `res.utr.pkl` as input).


__In this example__, the first command generated `cluster_wrt_CB.gene.pa.len.csv`, while the seconde command generated `author_cell_type.utr.pa.len.csv`. Both are in the output directory `/your-scape/examples/toy-example`.

The file looks like following:

```python
## This code is run in python
import pandas as pd

df = pd.read_csv("/your-scape/examples/toy-example/cluster_wrt_CB.gene.pa.len.csv")
df.head()
>>>
             gene_id cell_cluster  exp_length  num_pa
0  ENSG00000099194:1        cell1    8.555119       2

```

## Extract read count matrix

This step will generate a matrix count in which rows are PA sites and columns are cell barcodes.

```bash
## This code is run in bash

scape ex_pa_cnt_mat \
--output_dir /your-scape/examples/toy-example \
--res_pkl_file res.gene.pkl

scape ex_pa_cnt_mat \
--output_dir /your-scape/examples/toy-example \
--res_pkl_file res.utr.pkl
```

__INPUT__
- `output_dir`: Directory which was used in previous steps to save output by `prepare_input` and `infer_pa`.
- `res_pkl_file`: Name of res pickle file that contains PASs for calculating expected PA length. Its name will be included in the file name of final result.


__OUTPUT__

An tsv.gz file named `{res_pkl_file.cnt.tsv.gz}` is stored in `output_dir/`. It contains a count matrix in which
- row names are PAS, in format of __chrom:exact of position of pa site:std of position of pa site:strand:pa site ID:gene ID:utr ID__
- column names are cell barcode


__In this example__, the first command generated `res.gene.cnt.tsv.gz`, while the seconde command generated `res.utr.cnt.tsv.gz`. Both are in the output directory `/your-scape/examples/toy-example`.

The file looks like following:


```python
## This code is run in python
import pandas as pd

df = pd.read_csv("/your-scape/examples/toy-example/res.gene.cnt.tsv.gz", index_col="pa_info", sep="\t")
df.iloc[1:2,1:5]
>>>
                                         C1_AAACCTGAGAGCCTAG-1  C1_AAACCTGAGCCCGAAA-1  C1_AAACCTGAGTACGCGA-1  C1_AAACCTGAGTGGCACA-1
pa_info                                                                                                                            
10:100364805:50.0:+:2:ENSG00000099194:1                    0.0                    1.0                    0.0                    0.0
```